In [1]:
import numpy as np

In [2]:
def markov(P, x0, T, n):
    """
    P  : matrice de transition de la chaine de Markov
    x0 : loi de la donnée initiale
    T  : durée de la simulation
    n  : nombre de simulation
    """
    a, b = P.shape
    assert a==b
    assert a==len(x0)
    ini = np.random.rand(n)
    choix = np.random.random(size=(n, T+1))
    resultat = (a-1)*np.ones(shape=(n, T+1), dtype=np.int)
    Q = np.cumsum(P, axis=1)
    loi_ini = np.cumsum(x0)
    for i in range(n):
        for k in range(a):
            if ini[i] < loi_ini[k]:
                resultat[i, 0] = k
                break
    for i in range(n):
        for j in range(T):
            for k in range(a):
                if choix[i, j] < Q[resultat[i, j], k] :
                    resultat[i, j+1] = k
                    break
    return resultat
        

In [3]:
def stoch(N):
    P = np.random.rand(N,N)
    return P/np.sum(P, axis=1).reshape(N,1)


In [4]:
np.sum(stoch(5), axis=1)

array([ 1.,  1.,  1.,  1.,  1.])

In [5]:
def loi(N):
    res = np.random.rand(N)
    return res/np.sum(res)

In [6]:
np.sum(loi(5))

1.0

In [7]:
P, x0 = stoch(5), loi(5)
chaine = markov(P, x0, 10, 6)

In [8]:
chaine

array([[1, 3, 2, 2, 3, 2, 4, 0, 2, 3, 0],
       [0, 4, 4, 4, 4, 4, 1, 0, 0, 2, 0],
       [1, 3, 1, 1, 1, 3, 1, 0, 3, 2, 0],
       [4, 4, 2, 3, 0, 4, 0, 3, 1, 3, 1],
       [1, 3, 1, 1, 3, 1, 0, 1, 3, 0, 2],
       [1, 0, 2, 0, 2, 4, 0, 0, 2, 0, 0]])

In [9]:
P, x0 = stoch(50), loi(50)

In [10]:
%%time 
chaine = markov(P, x0, 1000, 300)

CPU times: user 3.52 s, sys: 4 ms, total: 3.52 s
Wall time: 3.52 s


In [11]:
%%prun
chaine = markov(P, x0, 1000, 500)

In [12]:
%load_ext line_profiler

In [13]:
%lprun -f markov markov(P, x0, 1000, 200)

In [14]:
%load_ext Cython

In [45]:
%%cython -a

import numpy as np
cimport numpy as np

def markov_cy(np.ndarray[np.float_t, ndim=2] P, np.ndarray[np.float_t, ndim=1] x0, int T, int n):
    """
    P  : matrice de transition de la chaine de Markov
    x0 : loi de la donnée initiale
    T  : durée de la simulation
    n  : nombre de simulation
    """
    cdef int a, b, l
    a = P.shape[0]
    b = P.shape[1]
    l = x0.shape[0]
    assert a==b
    assert a==l
    
    cdef np.ndarray[np.float_t, ndim=1] ini=np.random.rand(n)
    cdef np.ndarray[np.float_t, ndim=2] choix= np.random.rand(n, T+1)
    cdef np.ndarray[np.int_t, ndim=2] resultat = (a-1)*np.ones(shape=(n, T+1), dtype=np.int)
    cdef np.ndarray[np.float_t, ndim=2] Q = np.cumsum(P, axis=1)
    cdef np.ndarray[np.float_t, ndim=1] loi_ini = np.cumsum(x0)
    
    cdef int i, j, k
    for i in range(n):
        for k in range(a):
            if ini[i] < loi_ini[k]:
                resultat[i, 0] = k
                break
    for i in range(n):
        for j in range(T):
            for k in range(a):
                if choix[i, j] < Q[resultat[i, j], k] :
                    resultat[i, j+1] = k
                    break
    return resultat

In [46]:
%%time
chaine = markov_cy(P, x0, 1000, 300)

CPU times: user 20 ms, sys: 0 ns, total: 20 ms
Wall time: 22.9 ms
